In [174]:
import pandas as pd
import numpy as np

In [176]:
income_vars = ""
for i in range(11,18):
    i_padded = str(i).rjust(2, '0')
    income_value = f"B19001_0{i_padded}E"
    income_vars = income_vars + income_value + ","
    
educ_vars = ""
for i in range(1,17):
    i_padded = str(i).rjust(2, '0')
    educ_value = f"B15003_0{i_padded}E"
    educ_vars = educ_vars + educ_value + ","

'B15003_001E,B15003_002E,B15003_003E,B15003_004E,B15003_005E,B15003_006E,B15003_007E,B15003_008E,B15003_009E,B15003_010E,B15003_011E,B15003_012E,B15003_013E,B15003_014E,B15003_015E,B15003_016E,'

In [178]:
# percent of residents over 50k in income +
# percent in poverty
# percent people of color 
# percent less than hs diploma

acs_year = "2019"
base_url = f"https://api.census.gov/data/{acs_year}/acs/acs5"

variable_dict = {"B01001_001E": "total_population",
                 "B17020_001E": "poverty_status",
                 "B19001_001E": "total_pop_income",
                 "B01001H_001E": "white_non_hispanic_population"}


variable_names = f"NAME,B19001_001E,{income_vars}{educ_vars}B01001_001E,B01001H_001E,B01001_002E,B02001_001E"
block_group_code = "*"
state_code = "17"
county_code = "*"
geo_query = f"block%20group:{block_group_code}&in=state:{state_code}%20county:{county_code}"

query = f"{base_url}?get={variable_names}&for={geo_query}"

print(query)
census_df = pd.read_json(query)

https://api.census.gov/data/2019/acs/acs5?get=NAME,B19001_001E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,B19001_017E,B15003_001E,B15003_002E,B15003_003E,B15003_004E,B15003_005E,B15003_006E,B15003_007E,B15003_008E,B15003_009E,B15003_010E,B15003_011E,B15003_012E,B15003_013E,B15003_014E,B15003_015E,B15003_016E,B01001_001E,B01001H_001E,B01001_002E,B02001_001E&for=block%20group:*&in=state:17%20county:*


In [179]:
census_df.columns = census_df.iloc[0]
census_df = census_df[1:]


In [180]:

value_cols = census_df.filter(regex='B').columns

census_df[value_cols] = census_df[value_cols].apply(pd.to_numeric, errors='coerce', axis = 1)


In [181]:
census_df \
    .assign(n_over_50k = lambda x: x['B19001_011E'] + x['B19001_012E'] + x['B19001_013E'] + x['B19001_014E'] +
                                   x['B19001_015E'] + x['B19001_016E'] + x['B19001_017E'],
            total_income = lambda x: x['B19001_001E'],
            pct_over_50k = lambda x: x['n_over_50k'] / x['total_income'],
            pct_white_non_hispanic = lambda x: x['B01001H_001E'] / x['B01001_001E'],
            n_nohs = lambda x: x['B15003_002E'] + x['B15003_003E'] + x['B15003_004E'] + x['B15003_005E'] +
                               x['B15003_006E'] + x['B15003_007E'] + x['B15003_008E'] + x['B15003_009E'] +
                               x['B15003_010E'] + x['B15003_011E'] + x['B15003_012E'] + x['B15003_013E'] +
                               x['B15003_014E'] + x['B15003_015E'] + x['B15003_016E'],
            total_educ = lambda x: x['B15003_001E'],
            pct_nohs = lambda x: x['n_nohs'] / x['total_educ']) \
    .sort_values(by = "pct_over_50k")


,NAME,B19001_001E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,B19001_017E,B15003_001E,...,county,tract,block group,n_over_50k,total_income,pct_over_50k,pct_white_non_hispanic,n_nohs,total_educ,pct_nohs
4844,"Block Group 1, Census Tract 5009, St. Clair Co...",397.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,492.0,...,163,500900,1,0.0,397.0,0.0,NaN,218.0,492.0,0.443089
9321,"Block Group 2, Census Tract 20, Peoria County,...",5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,...,143,002000,2,0.0,5.0,0.0,NaN,0.0,23.0,0.000000
2473,"Block Group 1, Census Tract 9800, Johnson Coun...",3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1888.0,...,087,980000,1,0.0,3.0,0.0,NaN,907.0,1888.0,0.480403
9453,"Block Group 2, Census Tract 2, McLean County, ...",40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,113,000200,2,0.0,40.0,0.0,NaN,0.0,0.0,NaN
7917,"Block Group 2, Census Tract 7705, Cook County,...",118.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,206.0,...,031,770500,2,0.0,118.0,0.0,NaN,158.0,206.0,0.766990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7718,"Block Group 0, Census Tract 9900, Cook County,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,031,990000,0,0.0,0.0,NaN,NaN,0.0,0.0,NaN
8205,"Block Group 1, Census Tract 3817, Cook County,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,031,381700,1,0.0,0.0,NaN,NaN,0.0,0.0,NaN
8960,"Block Group 2, Census Tract 8704, Warren Count...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,187,870400,2,0.0,0.0,NaN,NaN,0.0,3.0,0.000000
9335,"Block Group 1, Census Tract 105, McDonough Cou...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,109,010500,1,0.0,0.0,NaN,NaN,0.0,9.0,0.000000


In [95]:
census_df

,B19001_001E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,NAME,B01001_001E,B01001_002E,B02001_001E,B17020_001E,B19001_001E,B19001_002E,B19001_003E,state,county,tract,block group
1,538,14,48,44,21,23,45,"Block Group 1, Census Tract 8089, Cook County,...",1408,628,1408,None,538,19,12,17,031,808900,1
2,774,31,60,77,75,73,107,"Block Group 3, Census Tract 8089, Cook County,...",2068,1046,2068,None,774,24,15,17,031,808900,3
3,453,48,37,41,9,65,31,"Block Group 1, Census Tract 8098, Cook County,...",1003,470,1003,None,453,17,30,17,031,809800,1
4,286,38,12,118,12,43,26,"Block Group 3, Census Tract 8101, Cook County,...",775,390,775,None,286,0,13,17,031,810100,3
5,207,0,40,15,10,61,0,"Block Group 4, Census Tract 8101, Cook County,...",506,294,506,None,207,0,0,17,031,810100,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9687,428,0,35,32,49,18,38,"Block Group 1, Census Tract 714, Cook County, ...",931,515,931,None,428,45,35,17,031,071400,1
9688,367,0,28,7,15,11,14,"Block Group 1, Census Tract 8001, Cook County,...",942,485,942,None,367,9,23,17,031,800100,1
9689,374,11,10,39,28,24,48,"Block Group 1, Census Tract 8013, Cook County,...",876,423,876,None,374,20,44,17,031,801300,1
9690,720,45,99,123,129,56,83,"Block Group 2, Census Tract 8039.01, Cook Coun...",1787,990,1787,None,720,12,0,17,031,803901,2


In [50]:
base_url

'https://api.census.gov/data/2019/acs/acs5'

In [38]:
cols = ['B01001_001E','B01001_002E']

census_df[cols] = census_df[cols].apply(pd.to_numeric, errors='coerce')

census_df.agg(np.sum)

0
NAME           Block Group 1, Census Tract 8089, Cook County,...
B01001_001E                                             12770631
B01001_002E                                              6272172
state          1717171717171717171717171717171717171717171717...
county         0310310310310310310310310310310310310310312032...
tract          8089008089008098008101008101008101008101008083...
block group    1313412131212115342312231121223143242215634342...
dtype: object

In [45]:
pd.read_json("https://api.census.gov/data/2019/acs/acs5/cprofile/variables.json").head(30)

,variables
CBSA,"{'label': 'Geography', 'group': 'N/A', 'limit'..."
CD,"{'label': 'Geography', 'group': 'N/A', 'limit'..."
COUNTY,"{'label': 'Geography', 'group': 'N/A', 'limit'..."
CP02PR_2014_001E,{'label': '2010-2014 Estimates!!HOUSEHOLDS BY ...
CP02PR_2014_002E,{'label': '2010-2014 Estimates!!HOUSEHOLDS BY ...
CP02PR_2014_003E,{'label': '2010-2014 Estimates!!HOUSEHOLDS BY ...
CP02PR_2014_004E,{'label': '2010-2014 Estimates!!HOUSEHOLDS BY ...
CP02PR_2014_005E,{'label': '2010-2014 Estimates!!HOUSEHOLDS BY ...
CP02PR_2014_006E,{'label': '2010-2014 Estimates!!HOUSEHOLDS BY ...
CP02PR_2014_007E,{'label': '2010-2014 Estimates!!HOUSEHOLDS BY ...
